# Online Job Posting-Similarity of Jobs

<b>Import Libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')

C:\Users\Aanchal\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aanchal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aanchal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<b>Loading the dataset

In [2]:
# read csv into a DataFrame
data = pd.read_csv('data job posts.csv',parse_dates= ['date'])
data.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\nJOB TITL...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\nc...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\nI...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\nJOB...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\nn...,NaN,2004,1,False
3,Manoff Group\nJOB TITLE: BCC Specialist\nPOSI...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\nPe...,NaN,23 January 2004\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\nJOB TITLE: Software D...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\n- CV; \n-...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


<b>Removing rows based on Jobpost & Title

In [3]:
df = data.drop_duplicates(['jobpost','Title'])
print("Removed {0} duplicates (based on jobpost + Title)".format(data.shape[0]-df.shape[0]))

Removed 109 duplicates (based on jobpost + Title)


In [4]:
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\nJOB TITL...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\nc...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\nI...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\nJOB...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\nn...,NaN,2004,1,False
3,Manoff Group\nJOB TITLE: BCC Specialist\nPOSI...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\nPe...,NaN,23 January 2004\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\nJOB TITLE: Software D...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\n- CV; \n-...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


<b>Selecting useful columns for analysis:

In [5]:
df1 = df[['Title','JobDescription','RequiredQual','JobRequirment']]
df1.head()

,Title,JobDescription,RequiredQual,JobRequirment
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...
1,Full-time Community Connections Intern (paid i...,NaN,- Bachelor's Degree; Master's is preferred;\n-...,NaN
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...
4,Software Developer,NaN,- University degree; economical background is ...,- Rendering technical assistance to Database M...


<b>Applying Data prep & Cleaning steps on all the 4 columns separately:

In [6]:
#Converting all four columns into string
df1['JobDescription'] = df1['JobDescription'].astype(str)
df1['RequiredQual'] = df1['RequiredQual'].astype(str)
df1['JobRequirment'] = df1['JobRequirment'].astype(str)
df1['Title'] = df1['Title'].astype(str)

<b>1)Tokenization

In [7]:
df1['JobDescription_token'] = df1['JobDescription'].map(word_tokenize)
df1['RequiredQual_token'] = df1['RequiredQual'].map(word_tokenize)
df1['JobRequirment_token'] = df1['JobRequirment'].map(word_tokenize)

<b>2)Remove punctuations and stopwords from the above tokens

In [9]:
# stopwords and punctuation removal
stop_words = set(stopwords.words('english'))
df1['JobDescription_token'] = df1['JobDescription_token'].apply(lambda x: [item for item in x if item not in stop_words])
df1['JobDescription_token'] = df1['JobDescription_token'].apply(lambda x: [item for item in x if item not in string.punctuation])
df1['RequiredQual_token'] = df1['RequiredQual_token'].apply(lambda x: [item for item in x if item not in stop_words])
df1['RequiredQual_token'] = df1['RequiredQual_token'].apply(lambda x: [item for item in x if item not in string.punctuation])
df1['JobRequirment_token'] = df1['JobRequirment_token'].apply(lambda x: [item for item in x if item not in stop_words])
df1['JobRequirment_token'] = df1['JobRequirment_token'].apply(lambda x: [item for item in x if item not in string.punctuation])

<b>3)Converting above into lowercase

In [10]:
# lowercase
df1['JobDescription_token'] = df1['JobDescription_token'].apply(lambda x: [item.lower() for item in x])
df1['RequiredQual_token'] = df1['RequiredQual_token'].apply(lambda x: [item.lower() for item in x])
df1['JobRequirment_token'] = df1['JobRequirment_token'].apply(lambda x: [item.lower() for item in x])

<b>4)Stemming

In [11]:
# stem
df1['JobDescription_token'] = df1['JobDescription_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
df1['RequiredQual_token'] = df1['RequiredQual_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])
df1['JobRequirment_token'] = df1['JobRequirment_token'].apply(lambda x: [PorterStemmer().stem(item) for item in x])

<b>5)Converting above list back to string

In [12]:
# convert list to string
df1['JobDescription_token_str'] = df1['JobDescription_token'].apply(lambda x: " ".join(x))
df1['RequiredQual_token_str'] = df1['RequiredQual_token'].apply(lambda x: " ".join(x))
df1['JobRequirment_token_str'] = df1['JobRequirment_token'].apply(lambda x: " ".join(x))

In [13]:
df1.head()


,Title,JobDescription,RequiredQual,JobRequirment,JobDescription_token,RequiredQual_token,JobRequirment_token,JobDescription_token_str,RequiredQual_token_str,JobRequirment_token_str
0,Chief Financial Officer,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\nindividu...",- Supervises financial management and administ...,"[ameria, invest, consult, compani, seek, chief...","[to, perform, job, success, individu, must, ab...","[supervis, financi, manag, administr, staff, i...",ameria invest consult compani seek chief finan...,to perform job success individu must abl perfo...,supervis financi manag administr staff includ ...
1,Full-time Community Connections Intern (paid i...,nan,- Bachelor's Degree; Master's is preferred;\n-...,nan,[nan],"[bachelor, 's, degre, master, 's, prefer, exce...",[nan],nan,bachelor 's degre master 's prefer excel skill...,nan
2,Country Coordinator,Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,"[public, outreach, strengthen, grow, network, ...","[degre, environment, relat, field, 5, year, re...","[work, countri, director, provid, environment,...",public outreach strengthen grow network enviro...,degre environment relat field 5 year relev exp...,work countri director provid environment infor...
3,BCC Specialist,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,"[the, lead, local, enhanc, develop, health, bc...","[advanc, degre, public, health, social, scienc...","[identifi, gap, knowledg, overse, inform, coll...",the lead local enhanc develop health bcc speci...,advanc degre public health social scienc commu...,identifi gap knowledg overse inform collect fi...
4,Software Developer,nan,- University degree; economical background is ...,- Rendering technical assistance to Database M...,[nan],"[univers, degre, econom, background, plu, exce...","[render, technic, assist, databas, manag, syst...",nan,univers degre econom background plu excel know...,render technic assist databas manag system rea...


<b>Similarity based on Tf-Idf

<b>TF-IDF Vectorisation

In [14]:
tfidf_model = TfidfVectorizer()

In [15]:
Title_tfidf = tfidf_model.fit_transform(df1['Title'])
JobDescription_tfidf = tfidf_model.fit_transform(df1['JobDescription_token_str'])
RequiredQual_tfidf = tfidf_model.fit_transform(df1['RequiredQual_token_str'])
JobRequirment_tfidf = tfidf_model.fit_transform(df1['JobRequirment_token_str'])

In [16]:
def find_similar_jobs_tfidf(index, top_n = 5):
    Title_tfidf_cosine = cosine_similarity(Title_tfidf[index:index+1], Title_tfidf).flatten()
    JobDescription_tfidf_cosine = cosine_similarity(JobDescription_tfidf[index:index+1], JobDescription_tfidf).flatten()
    RequiredQual_tfidf_cosine = cosine_similarity(RequiredQual_tfidf[index:index+1], RequiredQual_tfidf).flatten()
    JobRequirment_tfidf_cosine = cosine_similarity(JobRequirment_tfidf[index:index+1], JobRequirment_tfidf).flatten()
#Calculating overall similarity by taking avaerage of individual similarity
    average_cosine = (Title_tfidf_cosine + JobDescription_tfidf_cosine + RequiredQual_tfidf_cosine + JobRequirment_tfidf_cosine) / 4.0
    related_docs_indices = [i for i in average_cosine.argsort()[::-1] if i != index]
    return [(index, average_cosine[index]) for index in related_docs_indices][0:top_n]

In [17]:
df1.iloc[0]

Title                                                 Chief Financial Officer
JobDescription              AMERIA Investment Consulting Company is seekin...
RequiredQual                To perform this job successfully, an\nindividu...
JobRequirment               - Supervises financial management and administ...
JobDescription_token        [ameria, invest, consult, compani, seek, chief...
RequiredQual_token          [to, perform, job, success, individu, must, ab...
JobRequirment_token         [supervis, financi, manag, administr, staff, i...
JobDescription_token_str    ameria invest consult compani seek chief finan...
RequiredQual_token_str      to perform job success individu must abl perfo...
JobRequirment_token_str     supervis financi manag administr staff includ ...
Name: 0, dtype: object

<b>Calculating 5 similar jobs: 

In [18]:
for index, score in find_similar_jobs_tfidf(0):
    print(df1.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                 Chief Financial Officer
JobDescription              River Island is seeking a Chief Financial Offi...
RequiredQual                - Master's degree in Management, Finance or Ec...
JobRequirment               - Develop tools and systems to provide critica...
JobDescription_token        [river, island, seek, chief, financi, offic, r...
RequiredQual_token          [master, 's, degre, manag, financ, econom, at,...
JobRequirment_token         [develop, tool, system, provid, critic, financ...
JobDescription_token_str    river island seek chief financi offic respons ...
RequiredQual_token_str      master 's degre manag financ econom at least 5...
JobRequirment_token_str     develop tool system provid critic financi oper...
Name: 11223, dtype: object
SIMILARITY SCORE:  0.4898332490345752
Title                                                 Chief Financial Officer
JobDescription              Armenian Datacom Company (ADC) is seeking a hi...

In [20]:
for index, score in find_similar_jobs_tfidf(1):
    print(df1.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                 IT Teacher  (full time)
JobDescription                                                            nan
RequiredQual                - Degree in Computer Science, Information Tech...
JobRequirment                                                             nan
JobDescription_token                                                    [nan]
RequiredQual_token          [degre, comput, scienc, inform, technolog, rel...
JobRequirment_token                                                     [nan]
JobDescription_token_str                                                  nan
RequiredQual_token_str      degre comput scienc inform technolog relat dis...
JobRequirment_token_str                                                   nan
Name: 114, dtype: object
SIMILARITY SCORE:  0.6631658107959908
Title                       Non-paid part or full time Administrative Intern
JobDescription                                                           nan
Req

In [21]:
for index, score in find_similar_jobs_tfidf(2):
    print(df1.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                           Country Coordinator - Armenia
JobDescription              CENN - Caucasus Environmental NGO Network - is...
RequiredQual                1. Education: University education in environm...
JobRequirment               -  Write first hand articles, conduct intervie...
JobDescription_token        [cenn, caucasu, environment, ngo, network, loo...
RequiredQual_token          [1, educ, univers, educ, environment, scienc, ...
JobRequirment_token         [write, first, hand, articl, conduct, intervie...
JobDescription_token_str    cenn caucasu environment ngo network look coun...
RequiredQual_token_str      1 educ univers educ environment scienc relat f...
JobRequirment_token_str     write first hand articl conduct interview coll...
Name: 404, dtype: object
SIMILARITY SCORE:  0.45858797345562224
Title                                                     Country Coordinator
JobDescription              Veya Limited needs an experienced Business Man...


In [28]:
for index, score in find_similar_jobs_tfidf(3):
    print(df1.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                     Training Specialist and Coordinator
JobDescription              Working closely with Program Director and Chie...
RequiredQual                - Advanced degree in Public Administration, Po...
JobRequirment               - Represent the project as primary liaison wit...
JobDescription_token        [work, close, program, director, chief, parti,...
RequiredQual_token          [advanc, degre, public, administr, polit, scie...
JobRequirment_token         [repres, project, primari, liaison, program, p...
JobDescription_token_str    work close program director chief parti train ...
RequiredQual_token_str      advanc degre public administr polit scienc hum...
JobRequirment_token_str     repres project primari liaison program partner...
Name: 817, dtype: object
SIMILARITY SCORE:  0.16187681133303428
Title                                                       Health Specialist
JobDescription              Health Specialist will provide overall oversig...


In [29]:
for index, score in find_similar_jobs_tfidf(4):
    print(df1.iloc[index])
    print("SIMILARITY SCORE: ", score)

Title                                                      Software Developer
JobDescription                                                            nan
RequiredQual                - Over 2 years of experience in software devel...
JobRequirment               - Design and develop software solutions accord...
JobDescription_token                                                    [nan]
RequiredQual_token          [over, 2, year, experi, softwar, develop, stro...
JobRequirment_token         [design, develop, softwar, solut, accord, busi...
JobDescription_token_str                                                  nan
RequiredQual_token_str      over 2 year experi softwar develop strong know...
JobRequirment_token_str     design develop softwar solut accord busi requi...
Name: 14350, dtype: object
SIMILARITY SCORE:  0.5982009518237009
Title                                                      Software Developer
JobDescription                                                            nan